___
# Ciência dos Dados - PROJETO 2

___
## Nomes:  Gabriel Couto, Gabriel Miras e Mariana Abrantes
___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [1]:
import pandas as pd
import os
import random
import numpy as np
import re

In [2]:
#Carregar dados:
leitura = pd.ExcelFile('spamham2019(1).xlsx')
dados = pd.read_excel(leitura)

In [3]:
#SEPARAR EM TREINO E TESTE
treinoOrig=dados.sample(frac=0.75,random_state=200)
testeOrig=dados.drop(treinoOrig.index)

#Cria índices numéricos pros dataframes
treinoOrig.index = range(len(treinoOrig))
testeOrig.index = range(len(testeOrig))

#Criar novas tabelas pra n adulterar as originais que serão comparadas ao comparar as análises
teste, treino = testeOrig.copy(deep=True), treinoOrig.copy(deep=True)

#  Limpando a base de dados

#### 1) Para usar o mesmo filtro no teste e no treino criamos a função limparDados

In [4]:
#Função recebe linha, separa as palavras pelos espaços,  filtra e depois retorna uma lista palavras
def limparDados(line):
    padrao = r"[^a-z ]"
    palavrasFiltradas=[]

    linha = re.sub(padrao,' ',line.lower())
    palavras_linha = linha.split(' ')
    
    
    for i in palavras_linha:
        limp = re.sub(padrao,'',i)
        if len(limp)>1:
            palavrasFiltradas.append(limp)
    return(palavrasFiltradas)

##### 2) Criar tabela de frequências de cada palavra:
Dicionario tabela de frequências:<br>
&emsp; A &emsp; = nº de vezes que determinada palavra aparece em email Ham <br>
&emsp; B &emsp; = nº de vezes que determinada palavra aparece em email Spam <br>
&emsp; pPiA = P(P∩A) = prob de determinada palavra aparecer num email Ham <br>
&emsp; pPiB = P(P∩B) = prob de determinada palavra aparecer num email spam <br>
&emsp; pPcA = P(P|A) = prob de determinada palavra aparecer dado que o email é Ham <br>
&emsp; pPcB = P(P|B) = prob de determinada palavra aparecer dado que o email é Spam <br>

In [6]:
freq = {} #Tabela de frequências 

for numLinha in range(len(treino.Email)): 
    palavras_linha = limparDados(treino.loc[numLinha,'Email'])

    for palavra in palavras_linha:
        if palavra not in freq:
            freq[palavra] = {'A':0,'B':0,'pPiA':0,'pPiB':0,'pPcA':0,'pPcB':0}
        if treino.loc[numLinha,'Class'] == 'ham':
            freq[palavra]['A'] +=1 #A é ham
        else:
            freq[palavra]['B'] +=1

Outras variáveis:<br>
&emsp; pA = prob. de um email ser A <br>
&emsp; pB = prob. de um email ser B <br>
&emsp; pP = prob. de aparecer determinada palavra P <br>
&emsp; palavrasA = quantidade de palavras que aparecem nos emails A (Contando as repetidas)<br>
&emsp; palavrasB = quantidade de palavras que aparecem nos emails B (Contando as repetidas<br>
&emsp; totalPalavras = quantidade total de palavras que aparecem nos emails (Contando as repetidas)<br>
&emsp; palavrasDistintas = quantidade de palavras distintas que tem nos Emails (Não conta as repetidas)<br>

In [7]:
pA = (len(dados[dados.Class=='ham']))/(len(dados))
pB = (len(dados[dados.Class=='spam']))/(len(dados))
palavrasA,palavrasB = 0,0

for key,valor in freq.items():  
    palavrasA += valor['A']
    palavrasB += valor['B']
totalPalavras = palavrasA + palavrasB
palavrasDistintas = len(freq)



for palavra,valor in freq.items():
    #Prob de determinada palavra aparecer:
    freq[palavra]['pP'] = (valor['A']+valor['B'])/(totalPalavras)
    
    freq[palavra]['pPcA'] = (valor['A']+1)/(palavrasA+palavrasDistintas) #Laplace Smoothing
    freq[palavra]['pPcB'] = (valor['B']+1)/(palavrasB+palavrasDistintas) #Laplace Smoothing
        
    freq[palavra]['pPiA'] = freq[palavra]['pPcA']*pA
    freq[palavra]['pPiB'] = freq[palavra]['pPcB']*pB

Transformando a tabela num dataframe:

In [8]:
#Para melhorar a visualização da tabela de frequências colocamos ela num dataframe:
tabelapalavras = pd.DataFrame(freq).T
tabelapalavras.head()

,A,B,pP,pPcA,pPcB,pPiA,pPiB
no,228.0,57.0,0.004834,0.004239,0.003177,0.003671,0.000426
it,519.0,25.0,0.009227,0.009625,0.001424,0.008335,0.000191
not,310.0,13.0,0.005479,0.005757,0.000767,0.004985,0.000103
pride,2.0,0.0,0.000034,0.000056,0.000055,0.000048,0.000007
almost,11.0,0.0,0.000187,0.000222,0.000055,0.000192,0.000007


In [9]:
print('PalavrasA (com repet):',palavrasA,'| PalavrasB (com repet):',palavrasB, '| totalPalavras:',totalPalavras)
print('PalavrasDistintas (sem repet):',palavrasDistintas,'| pA:',pA,'| pB:',pB)

PalavrasA (com repet): 47361 | PalavrasB (com repet): 11595 | totalPalavras: 58956
PalavrasDistintas (sem repet): 6664 | pA: 0.8659368269921034 | pB: 0.13406317300789664


___
## 3. Classificador Naive-Bayes

#### 3) Classificador:

In [10]:
mensagens={}

for numLinha in range(len(teste.Email)):   
    mensagens[numLinha] = {'Email original':testeOrig.loc[numLinha,'Email'],
                           'Email Filtrado':" ".join(limparDados(teste.loc[numLinha,'Email'])),
                           'Classe original':teste.loc[numLinha,'Class'],
                            'pAcT':0,'pBcT':0,'lista pAcT':[], 'lista pBcT':[]}
    
    lAcT,lBcT=[],[]
    palavras_linha = limparDados(teste.loc[numLinha,'Email'])
    
    for palavra in palavras_linha:    
        if palavra in tabelapalavras.index:
            lAcT.append(tabelapalavras.loc[palavra,'pPcA'])
            lBcT.append(tabelapalavras.loc[palavra,'pPcB'])
        else:
            lAcT.append(1/(palavrasA +palavrasDistintas))
            lBcT.append(1/(palavrasB + palavrasDistintas))
    
    pAcT, pBcT = np.prod(lAcT) , np.prod(lBcT)

    mensagens[numLinha]['pAcT'] = pAcT*pA
    mensagens[numLinha]['pBcT'] = pBcT*pB
    mensagens[numLinha]['lista pAcT'] = lAcT
    mensagens[numLinha]['lista pBcT'] = lBcT

    if pAcT >= pBcT:
        mensagens[numLinha]['Classe calculada'] = 'ham'
    if pAcT < pBcT: 
        mensagens[numLinha]['Classe calculada'] = 'spam'

#Para melhorar a visualização dos emails colocamos ela num dataframe:
tabelaEmails = pd.DataFrame(mensagens).T
tabEmailsLimpo = tabelaEmails.loc[:,['Classe original','Classe calculada','Email original','Email Filtrado','pAcT', 'pBcT']]

a = tabelaEmails[tabelaEmails['Classe original'] != tabelaEmails['Classe calculada']].sort_values(by='pAcT',ascending=False)

tabEmailsLimpo.head()

,Classe original,Classe calculada,Email original,Email Filtrado,pAcT,pBcT
0,spam,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in wkly comp to win fa cup final tk...,3.78026e-93,4.6555e-73
1,ham,ham,Eh u remember how 2 spell his name... Yes i di...,eh remember how spell his name yes did he naug...,1.01765e-43,8.13083e-51
2,ham,ham,Fine if thatåÕs the way u feel. ThatåÕs the wa...,fine if that the way feel that the way its gota,3.89608e-30,1.82526e-39
3,ham,ham,I see the letter B on my car,see the letter on my car,7.56754e-17,6.73532e-21
4,ham,ham,Pls go ahead with watts. I just wanted to be s...,pls go ahead with watts just wanted to be sure...,1.17336e-44,2.39041e-50


In [11]:
VerHam, VerSpam, FalHam, FalSpam = 0,0,0,0
for numLinha in range(len(mensagens)):
    #Negativo Verdadeiro
    if (tabelaEmails.loc[numLinha,'Classe original'] == 'ham')&(tabelaEmails.loc[numLinha,'Classe calculada'] == 'ham'): 
        VerHam+=1
    #Positivos Verdadeiros
    if (tabelaEmails.loc[numLinha,'Classe original'] == 'spam')&(tabelaEmails.loc[numLinha,'Classe calculada'] == 'spam'):
        VerSpam+=1
    #Falso Negativo
    if (tabelaEmails.loc[numLinha,'Classe original'] == 'spam')&(tabelaEmails.loc[numLinha,'Classe calculada'] == 'ham'): 
        FalHam+=1
    #Falsos Positivos
    if (tabelaEmails.loc[numLinha,'Classe original'] == 'ham')&(tabelaEmails.loc[numLinha,'Classe calculada'] == 'spam'): 
        FalSpam+=1

#Positivo = ham | Negativo = spam 
print('Positivos Verdadeiros (Verdadeiros Spam): ',VerSpam)
print('Negativos Verdadeiros (Verdadeiros Ham): ',VerHam)
print('Falsos Positivos (Ham calculados como Spam):',FalSpam)
print('Falsos Negativos (Spam calculados como Ham): ',FalHam)
print()
print('Precisão de acerto dos Ham\'s:',round(100*VerHam/(VerHam+FalSpam),3),'%')
print('Precisão de acerto dos Spam\'s:',round(100*VerSpam/(VerSpam+FalHam),3),'%')

Positivos Verdadeiros (Verdadeiros Spam):  182
Negativos Verdadeiros (Verdadeiros Ham):  1154
Falsos Positivos (Ham calculados como Spam): 47
Falsos Negativos (Spam calculados como Ham):  10

Precisão de acerto dos Ham's: 96.087 %
Precisão de acerto dos Spam's: 94.792 %
